In [1]:
!pip install pandas
import pandas as pd

In [2]:
!pip install requests

In [3]:
import requests

#téléchargement des données 

def download_file(url, local_filename):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    return local_filename

download_file('https://datasets.imdbws.com/title.ratings.tsv.gz', 'title.ratings.tsv.gz')
download_file('https://datasets.imdbws.com/title.basics.tsv.gz', 'title.basics.tsv.gz')
download_file('https://datasets.imdbws.com/title.principals.tsv.gz', 'title.principals.tsv.gz')


'title.principals.tsv.gz'

In [5]:
import pandas as pd
#Types que nous allons attribuer aux colonnes de ratings
ratings_dtypes = {'tconst': str, 'averageRating': float, 'numVotes': int}

def load_tsv(file_name,dtype, na_values=None):
    return pd.read_csv(file_name, sep='\t', compression='gzip', dtype=dtype, na_values=na_values)

ratings = load_tsv(file_name='title.ratings.tsv.gz', dtype=ratings_dtypes)

In [6]:
#Types que nous allons attribuer aux colonnes de basics
basics_dtypes = {'tconst': str, 'primaryTitle': str, 'originalTitle': str, 'isAdult': int, 'startYear': int, 'runtimeMinutes': int, 'genres': str, 'endYear': str}



In [16]:
basics_dtypes = {'tconst': str, 'primaryTitle': str, 'originalTitle': str, 'isAdult': str, 'startYear': str,'runtimeMinutes': str,'genres': str, 'endYear': str}

basics = pd.read_csv('title.basics.tsv.gz', sep='\t', compression='gzip', dtype=basics_dtypes, na_values='\\N')
basics['isAdult'] = pd.to_numeric(basics['isAdult'], errors='coerce').astype('Int64')
basics['startYear'] = pd.to_numeric(basics['startYear'], errors='coerce').astype('Int64')



In [8]:
#Types que nous allons attribuer aux colonnes de principals  
principals_dtypes = {'tconst': str, 'ordering': int, 'ordering': str, 'job': str, 'characters': str}

In [9]:
principals = load_tsv(file_name='title.principals.tsv.gz',dtype=principals_dtypes)

In [17]:
movies_df = basics[(basics['titleType'] == 'movie') & (basics['startYear'] >= 2010)] #On ne garde que les informations concernant les films
movies_principals = pd.merge(movies_df, principals, on='tconst') #On effectue une jointure avec la table contenant les informations sur le casting 
actors_df = movies_principals[movies_principals['category'].isin(['actor', 'actress'])] #On retient uniquement les informations sur les acteurs 
actor_movie_counts = actors_df.groupby('nconst').size().reset_index(name='movie_count') 
top_actors = actor_movie_counts.sort_values(by='movie_count', ascending=False).head(10)
print(top_actors)


           nconst  movie_count
413     nm0000616          242
366597  nm8673980          154
6395    nm0103977          152
208563  nm3248232          150
197055  nm2794335          132
221078  nm3708709          130
3760    nm0043199          128
138888  nm1428724          117
289304  nm5724719          110
310699  nm6489058          105


In [18]:
#On reprend que les films
movies_df = basics[basics['titleType'] == 'movie']

#Jointure
merged_movies = pd.merge(movies_df, ratings, on='tconst')

#Séparation des genres
merged_movies['genres'] = merged_movies['genres'].str.split(',')
exploded_movies = merged_movies.explode('genres')

top_movies_by_genre = exploded_movies.groupby('genres').apply(lambda x: x.nlargest(5, 'averageRating')).reset_index(drop=True)

/tmp/ipykernel_345/2501005994.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_movies_by_genre = exploded_movies.groupby('genres').apply(lambda x: x.nlargest(5, 'averageRating')).reset_index(drop=True)


In [19]:
for genre, group in top_movies_by_genre.groupby('genres'):
    print(f"Top 5 movies in genre: {genre}")
    print(group[['primaryTitle', 'averageRating']])
    print("\n")


Top 5 movies in genre: Action
                        primaryTitle  averageRating
0  Once Upon A Time in Philly Part 2           10.0
1                          NON ROWDY            9.9
2           The Document: On the Run            9.8
3                       Dominic Bane            9.8
4                        Lai Jhakaas            9.8


Top 5 movies in genre: Adult
                  primaryTitle  averageRating
5           Girls Loving Girls           10.0
6            The Squirt Locker            9.7
7  Cheater Cheater Pussy Eater            9.7
8             Nicole's Revenge            9.6
9                    Alyx Star            9.6


Top 5 movies in genre: Adventure
                primaryTitle  averageRating
10             Mr. Local Man           10.0
11                  Fortnite           10.0
12          Buried in Tucson            9.8
13                 Karsmênia            9.8
14  The Document: On the Run            9.8


Top 5 movies in genre: Animation
                 